# Description 
This notebook mainly provides the code for answering the following research questions:

        4) How can formulaic flexibility be measured?
                a) How can similar formulas be identified and quantified?
                b) To what extent can formulaic language be predicted?

# Imports and settings

In [1]:
import pandas as pd

from transformers import pipeline
from transformers import AutoTokenizer  
from transformers import AutoModelForMaskedLM

In [2]:
from formutils import *

In [3]:
df_full_no_punct = pd.read_json("../data-push/0d-sampling/df-merged-nopunct.json", encoding="utf-8")

df_main = df_full_no_punct[df_full_no_punct["type"] == "main"]
df_main_sample = df_main.sample(n=700, random_state=42)

text = "".join(df_main["text"].to_list()).lower()
text_sample = "".join(df_main_sample["text"].to_list()).lower()

# Analysis

# What are the most common skip-grams?

In [4]:
n = 5
k = 2
lower = 1
candidates = scored_ngrams_to_dict(get_skip_grams_by_score(text_sample, n, k, lower))

In [5]:
regex_skip = re.compile(r"\s*\([^)]*\)") #matches any substring within (), including (), and any surrounding whitespace
regex_noskip = re.compile(r"[()]") #matches any () within the skip-gram, but no content inside ()

In [6]:
counts = {}
for key in candidates.keys():
    key_string = re.sub(regex_noskip, "", key).strip()
    key_string_no_brackets = re.sub(regex_skip, "", key).strip()
    counts[key] = {"with_skips": text.count(key_string), "without_skips": text.count(key_string_no_brackets)}

In [7]:
for key, nested_dict in dict(counts).items():
    for nested_key, nested_value in dict(nested_dict).items():
        if nested_value == 0:
            del counts[key][nested_key]
    if not counts[key]:
        del counts[key]

In [8]:
skip_probs_df = pd.DataFrame.from_dict(counts, orient="index").reset_index().dropna()
skip_probs_df = skip_probs_df.rename(columns={
    "index": "Formula",
    "with_skips": "with_skips_count",
    "without_skips": "without_skips_count"}
)
skip_probs_df["with_skips_count"] = skip_probs_df["with_skips_count"].astype(int)
skip_probs_df["without_skips_count"] = skip_probs_df["without_skips_count"].astype(int)

In [9]:
drop_candidates = [0, 1]
threshold = 1
top_k = 15

In [10]:
skip_probs_df_filtered = (
    skip_probs_df[skip_probs_df["without_skips_count"] / skip_probs_df["with_skips_count"] > threshold]
)

skip_probs_df_filtered = skip_probs_df_filtered.sort_values(["without_skips_count","with_skips_count"], 
    ascending=[False, False]
)

skip_probs_df_filtered = skip_probs_df_filtered[(
    ~skip_probs_df_filtered["with_skips_count"].isin(drop_candidates)) 
    & (~skip_probs_df_filtered["without_skips_count"].isin(drop_candidates))]

# skip_probs_df_filtered[:top_k].to_latex(
#     f"../data-push/4-flexibility/top_{n}_skip_{k}_options_wo_skips.tex", 
#     index=False
# )

skip_probs_df_filtered[:top_k]

,Formula,with_skips_count,without_skips_count
3,in sehent (lesent) oder horent lesen,109,295
23,hundert jar (vierzig) (jar) vnd darnach in,21,272
18,insigel an disen brief (ich) der,10,229
1,brief sehent (lesent) oder horent lesen,55,226
2,abt (herman) des gotzhus ze sant,32,196
11,disen brief sehent (lesent) oder horent,70,194
0,den die in sehent (horent) oder,29,189
16,die disen brief sehent (lesent) oder,69,184
33,tag (ze) (phingsten) in dem jar do,8,171
9,dem (apt) (des) gotzhus ze sant gallen,3,170


In [11]:
threshold = 1
skip_probs_df_filtered = (
    skip_probs_df[skip_probs_df["with_skips_count"]
    / skip_probs_df["without_skips_count"] > threshold]
)
skip_probs_df_filtered = skip_probs_df_filtered.sort_values(
    ["with_skips_count",
     "without_skips_count"],
    ascending=[False, False])

# skip_probs_df_filtered[:top_k].to_latex(
#     f"../data-push/4-flexibility/top_{n}_skip_{k}_options_w_skips.tex", 
#     index=False
# )

skip_probs_df_filtered[:top_k]

,Formula,with_skips_count,without_skips_count
580,do man (zalt) von gottes geburt,520,56
4836,vnd tun chunt allen (den) di,420,6
5445,in dem jar do (man) (zalt) von,368,23
21,fur mich vnd (fur) alle min,320,213
97,fur mich vnd fur (alle) min,320,79
157,fur mich (vnd) fur alle min,320,49
106,fur vns vnd (fur) alle vnser,307,148
68,fur vns vnd fur (alle) vnser,307,79
696,fur vns (vnd) fur alle vnser,307,27
10,mich vnd (fur) alle min erben,298,226


# How can string similarity show formulaic flexibility?

In [15]:
n = 6
k = 3
similarity_function = jaccard_similarity
lower_threshold = 0
upper_threshold = 1

similarity_function = jaccard_similarity
similar_sequences = get_similar_sequences(
    text, n, k, similarity_function, lower_threshold, upper_threshold
)

for common_ngram, sequences in similar_sequences.items():
    print(f"{common_ngram}:")
    for sequence, similarity in sequences[:5]:
        print(f"\t{sequence}")

do man zalt von gottes geburt:
	do man zalt von christes geburt
	do man zalt von gots geburt
	do man zalt von crists geburt
	lerers do man zalt von gottes
	do man zalt von gottes gebuirt
disen brief der geben ist ze:
	ist der brief ist geben ze
	brief der geben ist ze der
	disen brief der geben ist der
	der brief ist geben ze der
	versigelt der brief ist geben ze
an disen brief der geben ist:
	ist der brief ist geben an
	brief der geben ist an der
	an disen brief der brief geben
	geben an disen brief der geben
	disen brief der geben ist der


In [16]:
n = 7
k = 3
similarity_function = jaccard_similarity
lower_threshold = 0
upper_threshold = 1

similarity_function = jaccard_similarity
similar_sequences = get_similar_sequences(
    text, n, k, similarity_function, lower_threshold, upper_threshold
)

for common_ngram, sequences in similar_sequences.items():
    print(f"{common_ngram}:")
    for sequence, similarity in sequences[:5]:
        print(f"\t{sequence}")

do man zalt von gottes geburt druzehenhundert:
	do man zalt von cristus geburt druzehenhundert
	do man von cristus geburt zalt druzehenhundert
	iar do man zalt von gottes geburt
	do man zalt von gottes geburt drutzehenhundert
	statt do man zalt von gottes geburt
an disen brief der geben ist ze:
	an disen brief der geben ist an
	disen brief der geben ist an der
	disen brief der geben ist ze der
	geben an disen brief der geben ist
	an disen brief der ist geben an
in dem jar do man zalt von:
	waydhouen in dem jar do man zalt
	dem jar do man zalt von crists
	in dem iar do man zalt von
	ist in dem jar do man zalt
	dem jar do man zalt von cristi


In [17]:
n = 8
k = 3
similarity_function = jaccard_similarity
lower_threshold = 0
upper_threshold = 1

similarity_function = jaccard_similarity
similar_sequences = get_similar_sequences(
    text, n, k, similarity_function, lower_threshold, upper_threshold
)

for common_ngram, sequences in similar_sequences.items():
    print(f"{common_ngram}:")
    for sequence, similarity in sequences[:5]:
        print(f"\t{sequence}")

die disen brief ansehent lesent oder horent lesen:
	die disen brief lesent oder horent lesen die
	die disen brief ansehent oder horent lesen die
	die disen brief ansehent lesen oder horent lesen
	die die disen brief ansehent oder horent lesen
	die disen brief ansehent lesent oder horent lesent
den die disen brief ansehent lesent oder horent:
	den die disen brief lesent oder horent lesent
	die disen brief ansehent lesent oder horent lesent
	allen den die disen brief ansehent oder horent
	den die disen brief ansehent oder horent lesen
	tagallen den die disen brief ansehent oder horent
do man zalt von gottes geburt druzehenhundert jar:
	do man von cristus geburt zalt druzehenhundert jar
	statt do man zalt von gottes geburt druzehenhundert
	do man zalt von gottes geburt druzehenhundert vnd
	do man von cristes geburt zalt druzehenhundert jar
	jar do man zalt von cristus geburt druzehenhundert


# How well are formulas predicted?

In [4]:
model_basic = AutoModelForMaskedLM.from_pretrained(f"../models/custom/distilroberta-base-mhg-charter-mlm-v1")
tokenizer_basic = AutoTokenizer.from_pretrained(f"../models/custom/distilroberta-base-mhg-charter-mlm-v1")
fill_mask_basic = pipeline("fill-mask", model=model_basic, tokenizer=tokenizer_basic, top_k=10)

model_advanced = AutoModelForMaskedLM.from_pretrained(f"../models/custom/xlm-roberta-base-mhg-charter-mlm-v1")
tokenizer_advanced = AutoTokenizer.from_pretrained(f"../models/custom/xlm-roberta-base-mhg-charter-mlm-v1")
fill_mask_advanced = pipeline("fill-mask", model=model_advanced, tokenizer=tokenizer_advanced, top_k=10)

In [19]:
formula_1 = "do man zalt von gottes geburt druzehenhundert jar"
df = predict_tokens(formula_1, tokenizer_basic, fill_mask_basic, fill_mask_advanced)
df.to_latex(f"../data-push/4-flexibility/formula_1_prediction.tex", index=False)

header = pd.MultiIndex.from_tuples(
    [
        ("", "Mask"),
        ("Basic", "Prediction"),
        ("Basic", "Score"),
        ("Advanced", "Prediction"),
        ("Advanced", "Score"),
    ]
)

df.columns = header
#df.to_latex(f"../data-push/4-flexibility/formula_1_prediction.tex", index=False)

df

/tmp/ipykernel_1758/573494732.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(f"../data-push/4-flexibility/formula_1_prediction.tex", index=False)


Basic             Advanced          
              Mask Prediction     Score Prediction     Score
0               do         do  0.892775         do  0.725251
1              man        man  0.967569        man  0.999654
2             zalt         ne  0.135914        zal  0.201608
3              von        von  0.997175        von  0.990158
4           gottes     Christ  0.792006    Christi  0.372387
5           geburt        tag  0.559107     genade  0.666679
6  druzehenhundert       alle  0.419639        dru  0.150542
7              jar        jar  0.932495        jar  0.799794

In [20]:
formula_2 = "offenlich gehenkt an disen brief der geben ist"
df = predict_tokens(formula_2, tokenizer_basic, fill_mask_basic, fill_mask_advanced)
#df.to_latex(f"../data-push/4-flexibility/formula_2_prediction.tex", index=False)
df

,Mask,Basic Prediction,Basic Score,Advanced Prediction,Advanced Score
0,offenlich,ir,0.069277,gel,0.291192
1,gehenkt,",",0.720605,geben,0.254597
2,an,an,0.832260,an,0.994140
3,disen,dem,0.975774,den,0.886364
4,brief,brief,0.992967,brief,0.983780
5,der,Der,0.460779,der,0.885228
6,geben,da,0.442437,geben,0.994885
7,ist,w,0.200096,ist,0.920487


In [21]:
formula_3 = "disen brief sehent lesent oder horent lesen"
df = predict_tokens(formula_3, tokenizer_basic, fill_mask_basic, fill_mask_advanced)
#df.to_latex(f"../data-push/4-flexibility/formula_3_prediction.tex", index=False)
df

,Mask,Basic Prediction,Basic Score,Advanced Prediction,Advanced Score
0,disen,den,0.639299,sen,0.975926
1,brief,brief,0.988801,brief,0.968555
2,sehent,an,0.664602,f,0.840011
3,lesent,",",0.384960,lesen,0.738976
4,oder,",",0.412741,oder,0.941954
5,horent,hor,0.560960,horen,0.967698
6,lesen,les,0.967405,lesen,0.740588


# How well does the model capture formulaic flexibility?

In [22]:
formula_1 = "do man zalt von gottes geburt druzehenhundert jar"
formula_1_masked = "do man zalt von <mask> geburt druzehenhundert jar"

result = fill_mask_advanced(formula_1_masked)
result_df = pd.DataFrame(result, columns=["token_str", "score"])
result_df_sorted = result_df.sort_values(by="score", ascending=False)
result_df_sorted.reset_index(drop=True, inplace=True)
#result_df_sorted.to_latex(f"../data-push/4-flexibility/flexibility_1_prediction.tex", index=False)
result_df_sorted

,token_str,score
0,Christi,0.372387
1,Kristi,0.299412
2,Crist,0.181596
3,Christ,0.030053
4,gott,0.029909
5,krist,0.016921
6,christ,0.012666
7,Kristus,0.012313
8,got,0.009754
9,Christus,0.006520


In [23]:
formula_2 = "offenlich gehenkt an disen brief der geben ist"
formula_2_masked = "offenlich <mask> an disen brief der geben ist"


result = fill_mask_advanced(formula_2_masked)
result_df = pd.DataFrame(result, columns=["token_str", "score"])
result_df_sorted = result_df.sort_values(by="score", ascending=False)
result_df_sorted.reset_index(drop=True, inplace=True)
#result_df_sorted.to_latex(f"../data-push/4-flexibility/flexibility_2_prediction.tex", index=False)
result_df_sorted

,token_str,score
0,geben,0.254597
1,dar,0.103075
2,wider,0.063861
3,gehen,0.033142
4,och,0.032450
5,s,0.028206
6,in,0.024841
7,an,0.023536
8,gegeben,0.019831
9,k,0.017057


In [24]:
formula_3 = "disen brief sehent lesent oder horent lesen"
formula_3_masked = "disen brief sehent lesent oder <mask> lesen"

result = fill_mask_advanced(formula_3_masked)
result_df = pd.DataFrame(result, columns=["token_str", "score"])
result_df_sorted = result_df.sort_values(by="score", ascending=False)
result_df_sorted.reset_index(drop=True, inplace=True)
#result_df_sorted.to_latex(f"../data-push/4-flexibility/flexibility_3_prediction.tex", index=False)
result_df_sorted

,token_str,score
0,horen,0.967698
1,sint,0.002684
2,lesen,0.001864
3,hoe,0.001748
4,horren,0.001433
5,nut,0.001112
6,not,0.001022
7,wollen,0.000983
8,wil,0.000971
9,in,0.000781


# How well can models predict missing charter language?

In [25]:
# https://www.monasterium.net/mom/AT-OOeLA/GarstenOSB/1291_VI_09/charter
text_sample_1 = """" chavof ist geschehen, vnd hat er wernhart dem abbt vnd dem Goteshavs gesazt tvor rehten scherm des wingartens pvorgel, die hie
               genant sint, vlrichen den mveter Siboten hoentlin, herworten den Leitnoer vnd sich selben wernhart vnd sinen brvoder
               Hainrich am vrvar vnd habent di gelobet, swaz an ainem abge, daz sol der abbt vnd
               daz Goteshavs haben avf dem anderm vnd daz daz staetvnge vnd chraft habe, darvmbe
               gebe wir disen brief von wernhartes pete mit vnser states insigil versiglt vnd sint
               des zevge, di hie geschriben vnd genant sint: Di fvomfe pvorgel, di vor genant sint,
               vnd Ditmar der schachnoer, Hainrich der mvter, <mask> der pranter, hainrich vrowen tvoten
               svn vnd vlrich sin bruder, hainrich der habbrvonnoer, vlrich vnd hainrich <mask> <mask> wernhart
               des pranters svn vnd ander pidwer levte genvch. Diser brief ist gegewen, do von Christ gebv"""

In [26]:
df, full_prediction = get_recursive_predictions(text_sample_1, fill_mask_basic, fill_mask_advanced)
#df.to_latex(f"../data-push/4-flexibility/text_imputation_1.tex", index=False)
df

,Mask,Basic Prediction,Basic Score,Advanced Prediction,Advanced Score
0,<mask_105>,Ott,0.224434,Rudolf,0.114103
1,<mask_122>,s,0.321867,v,0.908269
2,<mask_123>,nd,0.298439,nd,0.871430


In [27]:
print(full_prediction)

" chavof ist geschehen, vnd hat er wernhart dem abbt vnd dem Goteshavs gesazt tvor rehten scherm des wingartens pvorgel, die hie
               genant sint, vlrichen den mveter Siboten hoentlin, herworten den Leitnoer vnd sich selben wernhart vnd sinen brvoder
               Hainrich am vrvar vnd habent di gelobet, swaz an ainem abge, daz sol der abbt vnd
               daz Goteshavs haben avf dem anderm vnd daz daz staetvnge vnd chraft habe, darvmbe
               gebe wir disen brief von wernhartes pete mit vnser states insigil versiglt vnd sint
               des zevge, di hie geschriben vnd genant sint: Di fvomfe pvorgel, di vor genant sint,
               vnd Ditmar der schachnoer, Hainrich der mvter, <Rudolf> der pranter, hainrich vrowen tvoten
               svn vnd vlrich sin bruder, hainrich der habbrvonnoer, vlrich vnd hainrich <v> <nd> wernhart
               des pranters svn vnd ander pidwer levte genvch. Diser brief ist gegewen, do von Christ gebv


In [28]:
# https://www.monasterium.net/mom/AT-StiASei/SeitenstettenOSB/1357_V_30/charter
text_sample_2 = """Ich Ott von Zintzndorff vergich mit dem offenn prief vnd tun chunt alln den leutn,
               di in sehnt oder hornt lesn, daz ich mit wolbedachtm mut vnd mit guetem rat vnd willn
               czu der czeit, do ich ez wol getun mochtt, den erbern herrn vnd fursten apt Englschalchn
               cze Seydensteten vnd sein gnants Gotshavs daselbs gancz vnd gar ledig sage vnd lazze
               aller der ansproch, die ich <mask> <mask> <mask> <mask> han auf seiner guter ains, des Schoephls lehn auf
               dem Graentleinsperg gnant in Groestner pharr gelegn, also, daz ich vnd alle mein erbn
               furbaz dhain ansprach dar vmb habn welln noch schulln in dhainn wegn, weder wenig
               noch vil. Vnd dar vmb czu eine steten vrchund gib ich dem vorgnantn Apt Englchalchn
               vnd seim wirdign Gotshaws cze Seydenstet den prief."""

In [29]:
df, full_prediction = get_recursive_predictions(text_sample_2, fill_mask_basic, fill_mask_advanced)
#df.to_latex(f"../data-push/4-flexibility/text_imputation_2.tex", index=False)
df

,Mask,Basic Prediction,Basic Score,Advanced Prediction,Advanced Score
0,<mask_67>,,0.127078,mit,0.370400
1,<mask_68>,g,0.062988,v,0.128786
2,<mask_69>,v,0.053827,v,0.050609
3,<mask_70>,t,0.115582,gehabt,0.550144


In [30]:
print(full_prediction)

Ich Ott von Zintzndorff vergich mit dem offenn prief vnd tun chunt alln den leutn,
               di in sehnt oder hornt lesn, daz ich mit wolbedachtm mut vnd mit guetem rat vnd willn
               czu der czeit, do ich ez wol getun mochtt, den erbern herrn vnd fursten apt Englschalchn
               cze Seydensteten vnd sein gnants Gotshavs daselbs gancz vnd gar ledig sage vnd lazze
               aller der ansproch, die ich <mit> <v> <v> <gehabt> han auf seiner guter ains, des Schoephls lehn auf
               dem Graentleinsperg gnant in Groestner pharr gelegn, also, daz ich vnd alle mein erbn
               furbaz dhain ansprach dar vmb habn welln noch schulln in dhainn wegn, weder wenig
               noch vil. Vnd dar vmb czu eine steten vrchund gib ich dem vorgnantn Apt Englchalchn
               vnd seim wirdign Gotshaws cze Seydenstet den prief.


In [9]:
# https://www.monasterium.net/mom/AT-StiAScho/SchottenOSB/1342_V_26/charter
text_sample_3 = """Ich Wernhart von Weiden vergich offenl <mask> <mask> <mask> <mask> <mask> meinem prvder Dietmaren 
                vnd auch allen leuten <mask> <mask> <mask> <mask> gvtem willen vnd mit verdochtem mut hon ze
                <mask> <mask> andert halbz viertail weins rethes perkreths, daz <mask> <mask> ze
               Pratheszen, vm siebencz phenninge vnd avch er <mask> <mask> schaffen schol ebichleichen
               allen sein frum, daz ich in noch niem <mask> daran schol irrn noch enmag. Dez gib ich
               im ze einem worn vrchvnd disen prieff. Daz ist geschehen, do man zalt von Cristez gepurd drewzehen hundert
               Jar darnoch in dem zwai vnd viertzistem iar, des nasten svntages noch phingsten."""

In [10]:
df, full_prediction = get_recursive_predictions(text_sample_3, fill_mask_basic, fill_mask_advanced)
#df.to_latex(f"../data-push/4-flexibility/text_imputation_3.tex", index=False)
df

,Mask,Basic Prediction,Basic Score,Advanced Prediction,Advanced Score
0,<mask_6>,ich,0.711016,",",0.595480
1,<mask_7>,v,0.400719,da,0.905321
2,<mask_8>,nd,0.146000,z,0.852875
3,<mask_9>,v,0.142962,ich,0.409291
4,<mask_10>,mit,0.554400,mit,0.170655
5,<mask_18>,mit,0.464363,",",0.896273
6,<mask_19>,nd,0.242895,da,0.314562
7,<mask_20>,mit,0.264256,ich,0.836183
8,<mask_21>,mit,0.944353,mit,0.978138
9,<mask_30>,a,0.133972,chauffe,0.785266


In [11]:
print(full_prediction)

Ich Wernhart von Weiden vergich offenl <,> <da> <z> <ich> <mit> meinem prvder Dietmaren 
                vnd auch allen leuten <,> <da> <ich> <mit> gvtem willen vnd mit verdochtem mut hon ze
                <chauffe> <gegeben> andert halbz viertail weins rethes perkreths, daz <da> <leit> ze
               Pratheszen, vm siebencz phenninge vnd avch er <da> <mit> schaffen schol ebichleichen
               allen sein frum, daz ich in noch niem <anders> daran schol irrn noch enmag. Dez gib ich
               im ze einem worn vrchvnd disen prieff. Daz ist geschehen, do man zalt von Cristez gepurd drewzehen hundert
               Jar darnoch in dem zwai vnd viertzistem iar, des nasten svntages noch phingsten.
